In [0]:
##################
# DATA IMPORTING #
##################

from os.path import exists

!wget -O cwe119_cgd.txt https://raw.githubusercontent.com/CGCL-codes/VulDeePecker/master/CWE-119/CGD/cwe119_cgd.txt

print()

with open("./cwe119_cgd.txt", "r") as cgd:
  print("Source: ",cgd.readline())

--2020-04-13 11:00:21--  https://raw.githubusercontent.com/CGCL-codes/VulDeePecker/master/CWE-119/CGD/cwe119_cgd.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17202057 (16M) [text/plain]
Saving to: ‘cwe119_cgd.txt’

cwe119_cgd.txt      100%[===================>]  16.40M  32.0MB/s    in 0.5s    

2020-04-13 11:00:21 (32.0 MB/s) - ‘cwe119_cgd.txt’ saved [17202057/17202057]


Source:  1 CVE-2010-1444/vlc_media_player_1.1.0_CVE-2010-1444_zipstream.c cfunc 449



In [13]:
import re
re_punctuation_string = '[()\s,/.\']'
programs=[]
program = []
statement=[]
labels = []
vocabulary = [] 
remove_token=['',' ',',','NULL','\n',';']
with open("./cwe119_cgd.txt", "r") as cgd:
  lines = cgd.readlines()
  for line in lines:
    statement=[token for token in re.split(re_punctuation_string,line)if token not in remove_token]
    #print(statement)
    if statement == ['---------------------------------']:
      if program[-1] == ['0']:
        labels.append(0)
      elif program[-1] == ['1']:
        labels.append(1)
      programs.append(program[1:-2])
      for s in program[1:]:
        for t in s:
           if t not in vocabulary:
             vocabulary.append(t)
      #print(program)
      program = []
    else:
      program.append(statement)

print(len(labels))


  
print(len(vocabulary))
   

39753
22631


In [0]:
# shuffle data
import random
randnum = random.randint(0,100)
random.seed(randnum)
random.shuffle(programs)
random.seed(randnum)
random.shuffle(labels)

In [65]:
#print(vocabulary[:20])
print(len([one for one in labels[:10000] if one == 1 ]))
print(len([zero for zero in labels[:10000] if zero == 0 ]))

2591
7409


In [66]:
print(len([one for one in labels[10000:15000] if one == 1 ]))
print(len([zero for zero in labels[10000:15000] if zero == 0]))


1292
3708


In [67]:
training_data = []
for i in range(10000):
  training_data.append((programs[i],labels[i]))
print(training_data[:2])

[([['ss_tc_root', '=', 'getenv', '"SS_TC_ROOT"'], ['size_dirpath', '=', 'strlen', 'ss_tc_root', '+', 'strlen', '"testData"', '+', '2;'], ['dirpath', '=', 'char*', 'malloc', 'size_dirpath', '*', 'sizeof', 'char'], ['sprintf', 'dirpath', '"%s', '%s"', 'ss_tc_root', '"testData"'], ['if', 'stat', 'dirpath', '&st', '==', '-1', '{'], ['retval', '=', 'mkdir', 'dirpath', '0700'], ['size_filepath', '=', 'strlen', 'dirpath', '+', 'strlen', '"logfile', 'txt"', '+', '2;']], 0), ([['data', '=', 'NULL;'], ['goodG2BSource', 'data'], ['void', 'goodG2BSource', 'wchar_t', '*', '&data'], ['data', '=', 'new', 'wchar_t[100];'], ['data[0]', '=', 'L', '\\0'], ['goodG2BSource', 'data'], ['wchar_t', 'source[100];'], ['wmemset', 'source', 'L', 'C', '100-1'], ['source[100-1]', '=', 'L', '\\0']], 0)]


In [68]:
eval_data=[]
for i in range(10000,15000):
  eval_data.append((programs[i],labels[i]))
print(eval_data[:2])

[([['data', '=', 'new', 'char[10];'], ['char', 'source[10+1]', '=', 'SRC_STRING;']], 1), ([['data', '=', 'new', 'wchar_t[10];'], ['wchar_t', '*', 'data', '=', 'CWE122_Heap_Based_Buffer_Overflow__cpp_CWE193_wchar_t_cpy_68_badData;'], ['wchar_t', 'source[10+1]', '=', 'SRC_STRING;']], 1)]


In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#look up table  
#using one-hot embedding
vocabulary_size = len(vocabulary)
def look_up_table(word_idx):
  x = torch.zeros(vocabulary_size).long()
  x[word_idx] = 1.0
  return x
word2idx = {w: idx +1 for (idx, w) in enumerate(vocabulary)}
idx2word = {idx+1: w for (idx, w) in enumerate(vocabulary)}
print(look_up_table(word2idx['1']))

tensor([0, 0, 0,  ..., 0, 0, 0])


In [70]:
context_size = 20
def get_programcontext(program):
  id = 0
  contexts =torch.zeros(context_size,1).long()
  for p in program:
    for t in p:
      if id < context_size:
        contexts[id]= word2idx[t]
        id = id +1
      else:
        return contexts
  return contexts
get_programcontext(programs[4920])

tensor([[  615],
        [  352],
        [  855],
        [  315],
        [    5],
        [  285],
        [  315],
        [    5],
        [ 3009],
        [13699],
        [  615],
        [13535],
        [    5],
        [  669],
        [    0],
        [    0],
        [    0],
        [    0],
        [    0],
        [    0]])

In [71]:


torch.manual_seed(1)
embedding_dim =15
class ProgrammingML(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(ProgrammingML, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.conv1 = nn.Conv2d(1, 3, 2)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 2)
    self.fc1 = nn.Linear(16 * 5 * 5, 2)

    self.linear1 = nn.Linear(embedding_dim*context_size,10)
    self.linear2 = nn.Linear(10,2)
  def forward(self, inputs):
    #print(inputs.size())
    embeds = self.embeddings(inputs.view(-1))
    #print(embeds)
    out = F.relu(self.linear1(embeds.view(-1)))
    out = F.softmax(self.linear2(out),dim=0)
    return out


losses = []
loss_function = nn.BCELoss()
model = ProgrammingML(vocabulary_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
  losses = 0
  for program, label in training_data:
   # print(program)
   # print(label)
   if label ==1:
     out_labels = torch.FloatTensor([0,1])
   else:
     out_labels  = torch.FloatTensor([1,0])

   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(out.size())
   #print(f'Out {out}')
   #print(out_labels)
   loss = loss_function(out, out_labels )
   #print(f'Loss {loss.item()}')
   loss.backward()
   optimizer.step()
   #print(f'Loss {loss.item()}')
   losses +=loss.item()
  print(f'Loss at epo {epoch}: {losses/len(training_data)}')

Loss at epo 0: 0.4403967569380693
Loss at epo 1: 0.33170990852703663
Loss at epo 2: 0.2784942467651442
Loss at epo 3: 0.2468668150356145
Loss at epo 4: 0.2260661949796362
Loss at epo 5: 0.2098709678972226
Loss at epo 6: 0.19885025114645813
Loss at epo 7: 0.1898927681236528
Loss at epo 8: 0.18199425340939737
Loss at epo 9: 0.17669907976739063
Loss at epo 10: 0.17117849828815696
Loss at epo 11: 0.16730360337942435
Loss at epo 12: 0.16468833964268786
Loss at epo 13: 0.16211086302018698
Loss at epo 14: 0.1595430445607416
Loss at epo 15: 0.15744020290338673
Loss at epo 16: 0.1556066154135405
Loss at epo 17: 0.15400628118536444
Loss at epo 18: 0.15191232715043063
Loss at epo 19: 0.15065191360464777


In [73]:
with torch.no_grad():
  accurate = 0
  for program, label in eval_data:
   # print(program)
   # print(label)
   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(f'{out}:{label}')
   if(out[0]>0.5):
     if(label == 0):
       accurate =accurate+1
   elif(out[1]>0.5):
     if(label == 1):
       accurate = accurate+1
  print(f'Accuracy: {accurate/len(eval_data)}')   

Accuracy: 0.8546
